---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
def answer_one():
        
    G_df = pd.read_table('Employee_Movie_Choices.txt',names = ['employees','movies']) # load data
    G_df = G_df.iloc[1:]
    G = nx.Graph()
    G.add_nodes_from(G_df['employees'],bipartite = 0)
    G.add_nodes_from(G_df['movies'],bipartite = 1)
    edges = [(a,b) for a,b in G_df.values]
    G.add_edges_from(edges)

    return G

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [3]:
def answer_two():
    G = answer_one()
    types = ('employee','movie')
    employee = list(employees)
    movie = list(movies)
    nodes = np.hstack([employee,movie])
    node_type = pd.Series(nodes).apply(lambda x:types[0] if x in employee else types[1])
    type_dict = dict(zip(nodes,node_type))
    nx.set_node_attributes(G,'type',type_dict)    
    return G

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [4]:
def answer_three():
    
    return bipartite.projected_graph(answer_two(),employees)

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [5]:
def answer_four():
    # created projected graph
    G = answer_two()
    G_weighted = bipartite.weighted_projected_graph(G,employees)

    # get data
    common_movies = nx.get_edge_attributes(G_weighted,'weight')
    pairs = list(common_movies.keys())
    common_nums = list(common_movies.values())

    # read relationship table
    R = pd.read_table('Employee_Relationships.txt',header=None)

    # convert data into dataframe
    pairs_a = pd.Series([a[0] for a in pairs])
    pairs_b = pd.Series([b[1] for b in pairs])
    data = {0:np.hstack([pairs_a,pairs_b]),1:np.hstack([pairs_b,pairs_a]),'movies':np.hstack([common_nums,common_nums])}
    G_weighted_df = pd.DataFrame(data = data)

    # merge two dataframes and find correlations
    df = pd.merge(R,G_weighted_df,how='left',left_on=[0,1],right_on = [0,1])
    df.fillna(0,inplace = True)
     
    return df.corr().iloc[0][1] # return correlation